This jupyter notebook proves [Theorem 2.15](https://joyofcryptography.com/pdf/chap2.pdf#page=18) from [The Joy of Cryptography](https://joyofcryptography.com/). Namely, that if a symmetric encryption scheme has one-time uniform ciphertexts, then it is also one-time secure.

# Definitions

First, we write a primitive definition for symmetric encryption schemes.

In [1]:
Primitive SymEnc(Set MessageSpace, Set CiphertextSpace, Set KeySpace) {
    Set Message = MessageSpace;
    Set Ciphertext = CiphertextSpace;
    Set Key = KeySpace;

    Key KeyGen();

    Ciphertext Enc(Key k, Message m);

    Message? Dec(Key k, Ciphertext c);
}

Next, we write a pair of games that model the one-time uniform ciphertexts property.

In [2]:
Game Real(SymEnc E) {
    E.Ciphertext CTXT(E.Message m) {
        E.Key k = E.KeyGen();
        E.Ciphertext c = E.Enc(k, m);
        return c;
    }
}

Game Random(SymEnc E) {
    E.Ciphertext CTXT(E.Message m) {
        E.Ciphertext c <- E.Ciphertext;
        return c;
    }
}

export as OneTimeUniformCiphertexts;

And a game that models the one-time security property.

In [3]:
Game Left(SymEnc E) {
    E.Ciphertext Eavesdrop(E.Message mL, E.Message mR) {
        E.Key k = E.KeyGen();
        E.Ciphertext c = E.Enc(k, mL);
        return c;
    }
}

Game Right(SymEnc E) {
    E.Ciphertext Eavesdrop(E.Message mL, E.Message mR) {
        E.Key k = E.KeyGen();
        E.Ciphertext c = E.Enc(k, mR);
        return c;
    }
}

export as OneTimeSecrecy;


# Proof

Our first reduction, `R1`, uses the real `OneTimeUniformCiphertexts CTXT` oracle to perfectly mimic the behaviour of the left `OneTimeSecrey` game. Replacing the real oracle with the random oracle by the assumption that `proofE` has one-time uniform ciphertexts then lets us hop to the second reduction `R2`. The only syntactic change between `R1` and `R2` is the argument passed to the CTXT oracle, which is not used whatsoever when interacting with the random challenger. Then, hopping to use the real challenger yields a game which is codewise equivalent to the right `OneTimeSecrecy` game, showing that `proofE` is one-time secure.

In [4]:
Reduction R1(SymEnc se) compose OneTimeUniformCiphertexts(se) against OneTimeSecrecy(se).Adversary {
    se.Ciphertext Eavesdrop(se.Message mL, se.Message mR) {
        return challenger.CTXT(mL);
    }
}

Reduction R2(SymEnc se2) compose OneTimeUniformCiphertexts(se2) against OneTimeSecrecy(se2).Adversary {
    se2.Ciphertext Eavesdrop(se2.Message mL, se2.Message mR) {
        return challenger.CTXT(mR);
    }
}

proof:

let:

    Set MessageSpace;
    Set CiphertextSpace;
    Set KeySpace;

    SymEnc proofE = SymEnc(MessageSpace, CiphertextSpace, KeySpace);

assume:

    OneTimeUniformCiphertexts(proofE);

theorem:
    OneTimeSecrecy(proofE);

games:

    OneTimeSecrecy(proofE).Left against OneTimeSecrecy(proofE).Adversary;

    // Codewise-equivalency
    OneTimeUniformCiphertexts(proofE).Real compose R1(proofE) against OneTimeSecrecy(proofE).Adversary;

    // By assumption
    OneTimeUniformCiphertexts(proofE).Random compose R1(proofE) against OneTimeSecrecy(proofE).Adversary;

    // mL argument is not used in R
    OneTimeUniformCiphertexts(proofE).Random compose R2(proofE) against OneTimeSecrecy(proofE).Adversary;

    // By assumption
    OneTimeUniformCiphertexts(proofE).Real compose R2(proofE) against OneTimeSecrecy(proofE).Adversary;

    // Codewise equivalency
    OneTimeSecrecy(proofE).Right against OneTimeSecrecy(proofE).Adversary;
